In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cv2
import math
import os,sys
import scipy.ndimage
import time
import scipy
from numpy.linalg import det, lstsq, norm
from functools import cmp_to_key

In [3]:
'''
首先，定义一个CSift类，用于传递SIFT所需的多个参数
'''
class  CSift: 
	def __init__(self,num_octave,sigma): 
		'''
		调用CSift类的时候，除初始化的一般参数外，还需要传入三个参数：
		num_octave: 高斯金字塔的组数
		sigma: 标准差，每一层对应的都不同
		'''
		self.sigma = sigma	# 初始尺度因子
		self.num_octave = 3 # 组数，后续重新计算
		'''
		以下参数为常量
		'''
		self.num_scale = 3 # 根据Lowe建议选择
		self.contrast_t = 0.04 # 弱响应阈值
		self.eigenvalue_r = 10 # hessian矩阵特征值的比值阈值
		self.scale_factor = 1.5 # 求取方位信息时的尺度系数
		self.radius_factor = 3 # 被采样率
		self.num_bins = 36 # 极值点方位方向划分
		self.peak_ratio = 0.8 # 极值点方向分配时，辅方向的权重

1.构建尺度空间：图像降采样+高斯模糊

Lowe论文原文：Of course, if we pre-smooth the image before extrema detection, we are effectively discarding the highest spatial frequencies. Therefore, to make full use of the input, the image can be expanded to <u>create more sample points</u> than were present in the original. We **double the size of the input image** using <u>*linear interpolation*</u> prior to building the first level of the pyramidamid.

首先通过cv2.resize方法，对原始图像进行2x缩放+高斯模糊，作为高斯金字塔的底层图像。目的是充分利用图像的空间结构信息。

效果：最终匹配得到的特征点对数量，提升四倍之多！

注：cv2.resize插值方法：
+ INTER_NEAREST 最近邻插值
+ INTER_LINEAR 双线性插值（默认设置）
+ INTER_AREA 使用像素区域关系进行重采样。
+ INTER_CUBIC 4x4像素邻域的双三次插值
+ INTER_LANCZOS4 8x8像素邻域的Lanczos插值



In [ ]:
# 获得高斯金字塔的底层图像
def get_base_img(img_src,sigma,cam_blur=0.5):
    '''
    img_src:输入原始图像
    sigma:尺度因子
    cam_blur: Lowe在论文中提出,原始图像的最小原始模糊sigma=0.5,防止混叠所需达到的最小值
    在这一步操作中，进行高斯模糊操作时需要注意折算cam_blur的效果（根据高斯模糊的性质）
    '''
    sigma_cor = np.sqrt(sigma**2 - (2*cam_blur)**2) # 2x缩放后的图像有2x的像素间隔，所以cam_blur也为2x
    img = img_src.copy()
    img = cv2.resize(img,dsize=None,fx=2,fy=2,interpolation=cv2.INTER_LINEAR) # cv2.resize 2x缩放
    img = cv2.GaussianBlur(img,ksize=None,sigmaX=sigma_cor,sigmaY=sigma_cor) # 对缩放后图像进行高斯模糊
    # 以折算后的尺度因子生成高斯卷积核，cv2.GaussianBlur方法调用了:
    # 1. cv2.GetGaussianKernal，分别生成两个一维高斯卷积核KernalX、KernalY；
    # 2. cv2.sepFilter2D，用kernalX对图像行做卷积，KernalY对图像列做卷积；
    # 最后对图像做归一化处理。
    return img # 得到底层图像

In [4]:
# 计算高斯金字塔的组数
def get_numOfOctave(img):
	num = round(np.log(min(img.shape[0],img.shape[1]))/np.log(2))-1 
	# o = log{2}{min(x,y)}-t，其中t=log{2}{min(x,y)(顶层图像)}
	# 本例中底层的图像尺寸为1220*1880，顶层图像尺寸为2*4, 因此t=1
	return num

In [ ]:
# 构建高斯金字塔
def construct_octave(img_src,s,sigma):
    '''
    s: 对单个octave的划分，对图像进行高斯模糊的期望次数
    '''
    octave = []
    octave.append(img_src)
    k = 2**(1/s) # octave的每个划分之间的尺度系数，保证顶层和底层的2x尺度关系
    for i in range(1,s+3): 
        '''
        这里进行了s+3次尺度滤波，因为Lowe认为额外的三次滤波的意义，是"so that final extrema detection covers a complete octave."
        但是3从何而来?s个极值点检测结果需要s+2个差分层(头尾各+1)，需要(s+2)+1个高斯滤波层。
        可以发现，进行到s+1次滤波时，对应的sigma就是2x本octave的sigma_0，正好是下一octave的sigma_0
        根据Lowe在原文中给出的建议，选取s=3
        '''
        img = octave[-1].copy()
        cur_sigma = k**i*sigma # 当前尺度 
        pre_sigma = k**(i-1)*sigma # 前一图像尺度
        true_sigma = np.sqrt(cur_sigma**2-pre_sigma**2) # 折算后本层对应尺度因子
        blur_img = cv2.GaussianBlur(img,ksize=None,sigmaX=true_sigma,sigmaY=true_sigma)
        octave.append(blur_img)
    return octave

def construct_gaussian_pyramid(img_src,sift:CSift):
    '''
    高斯金字塔的部分关键参数，通过CSift类传递
    '''
    pyramid=[]
    img_base = img_src.copy()
    for i in range(sift.num_octave):#共计构建octave组
        octave = construct_octave(img_base,sift.num_scale,sift.sigma) 
        pyramid.append(octave)
        img_base = octave[-3]#倒数第三层的尺度与下一组的初始尺度相同，对该层进行降采样，作为下一组的图像输入
        img_base = cv2.resize(img_base,(int(img_base.shape[1]/2),int(img_base.shape[0]/2)),interpolation=cv2.INTER_NEAREST)
    return pyramid

In [5]:
def do_match(img_src1,kp1,des1,img_src2,kp2,des2,embed=1,pt_flag=0,MIN_MATCH_COUNT = 10):
	## 1. 对关键点进行匹配 ##
	FLANN_INDEX_KDTREE = 0
	index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
	search_params = dict(checks=50)
	flann = cv2.FlannBasedMatcher(index_params, search_params)
	des1, des2 = np.array(des1).astype(np.float32), np.array(des2).astype(np.float32)#需要转成array
	matches = flann.knnMatch(des1, des2, k=2)  # matches为list，每个list元素由2个DMatch类型变量组成,分别是最邻近和次邻近点

	good_match = []
	for m in matches:
		if m[0].distance < 0.7 * m[1].distance:  # 如果最邻近和次邻近的距离差距较大,则认可
			good_match.append(m[0])
	## 2. 将2张图画在同一张图上 ##
	img1 = img_src1.copy()
	img2 = img_src2.copy()
	h1, w1 = img1.shape[0],img1.shape[1]
	h2, w2 = img2.shape[0],img2.shape[1]
	new_w = w1 + w2
	new_h = np.max([h1, h2])
	new_img =  np.zeros((new_h, new_w,3), np.uint8) if len(img_src1.shape)==3 else  np.zeros((new_h, new_w), np.uint8)
	h_offset1 = int(0.5 * (new_h - h1))
	h_offset2 = int(0.5 * (new_h - h2))
	if len(img_src1.shape) == 3:
		new_img[h_offset1:h_offset1 + h1, :w1,:] = img1  # 左边画img1
		new_img[h_offset2:h_offset2 + h2, w1:w1 + w2,:] = img2  # 右边画img2
	else:
		new_img[h_offset1:h_offset1 + h1, :w1] = img1  # 左边画img1
		new_img[h_offset2:h_offset2 + h2, w1:w1 + w2] = img2  # 右边画img2
	##3. 两幅图存在足够的匹配点，两幅图匹配成功，将匹配成功的关键点进行连线 ##
	if len(good_match) > MIN_MATCH_COUNT:
		src_pts = []
		dst_pts = []
		mag_err_arr=[]
		angle_err_arr=[]
		for m in good_match:
			if pt_flag==0:#point是百分比
				src_pts.append([kp1[m.queryIdx].pt[0] * img1.shape[1], kp1[m.queryIdx].pt[1] * img1.shape[0]])#保存匹配成功的原图关键点位置
				dst_pts.append([kp2[m.trainIdx].pt[0] * img2.shape[1], kp2[m.trainIdx].pt[1] * img2.shape[0]])#保存匹配成功的目标图关键点位置
			else:
				src_pts.append([kp1[m.queryIdx].pt[0], kp1[m.queryIdx].pt[1]])  # 保存匹配成功的原图关键点位置
				dst_pts.append([kp2[m.trainIdx].pt[0], kp2[m.trainIdx].pt[1]])  # 保存匹配成功的目标图关键点位置

			mag_err = np.abs(kp1[m.queryIdx].response - kp2[m.trainIdx].response) / np.abs(kp1[m.queryIdx].response )
			angle_err = np.abs(kp1[m.queryIdx].angle - kp2[m.trainIdx].angle)
			mag_err_arr.append(mag_err)
			angle_err_arr.append(angle_err)

		if embed!=0 :#若图像2是图像1内嵌入另一个大的背景中，则在图像2中，突出显示图像1的边界
			M = cv2.findHomography(np.array(src_pts), np.array(dst_pts), cv2.RANSAC, 5.0)[0]  # 根据src和dst关键点，寻求变换矩阵
			src_w, src_h = img1.shape[1], img1.shape[0]
			src_rect = np.array([[0, 0], [src_w - 1, 0], [src_w - 1, src_h - 1], [0, src_h - 1]]).reshape(-1, 1, 2).astype(
				np.float32)  # 原始图像的边界框
			dst_rect = cv2.perspectiveTransform(src_rect, M)  # 经映射后，得到dst的边界框
			img2 = cv2.polylines(img2, [np.int32(dst_rect)], True, 255, 3, cv2.LINE_AA)  # 将边界框画在dst图像上，突出显示
			if len(new_img.shape) == 3:
				new_img[h_offset2:h_offset2 + h2, w1:w1 + w2,:] = img2  # 右边画img2
			else:
				new_img[h_offset2:h_offset2 + h2, w1:w1 + w2] = img2  # 右边画img2

		new_img = new_img if len(new_img.shape) == 3 else  cv2.cvtColor(new_img, cv2.COLOR_GRAY2BGR)
		# 连线
		for pt1, pt2 in zip(src_pts, dst_pts):
			cv2.line(new_img, tuple(np.int32(np.array(pt1) + [0, h_offset1])),
					 tuple(np.int32(np.array(pt2) + [w1, h_offset2])), color=(0, 0, 255))
	return new_img


调用OpenCV库中的cv2.SIFT.create方法实现SIFT

In [6]:
if __name__ == '__main__':
	MIN_MATCH_COUNT = 10
	sift = CSift(num_octave=4,num_scale=3,sigma=1.6)
	img_src1 = cv2.imread('D:/Projects/SIFT Matching/remote_sensing_pair/image1.bmp',-1)
	#img_src1 = cv2.resize(img_src1, (0, 0), fx=.25, fy=.25)
	img_src2 = cv2.imread('D:/Projects/SIFT Matching/remote_sensing_pair/image2.bmp', -1)
	#img_src2 = cv2.resize(img_src2, (0, 0), fx=.5, fy=.5)
	# 2. 使用opencv自带sift算子
	sift.num_octave = get_numOfOctave(img_src1)
	opencv_sift = cv2.SIFT.create(nfeatures=None, nOctaveLayers=sift.num_octave,
								  contrastThreshold=sift.contrast_t, edgeThreshold=sift.eigenvalue_r, sigma=sift.sigma)
	kp1 = opencv_sift.detect(img_src1)
	kp1,des1 = opencv_sift.compute(img_src1,kp1)

	sift.num_octave = get_numOfOctave(img_src2)
	opencv_sift = cv2.SIFT.create(nfeatures=None, nOctaveLayers=sift.num_octave,
								  contrastThreshold=sift.contrast_t, edgeThreshold=sift.eigenvalue_r, sigma=sift.sigma)
	kp2 = opencv_sift.detect(img_src2)
	kp2, des2 = opencv_sift.compute(img_src2, kp2)
	pt_flag = 1

	# 3. 做匹配
	reu_img = do_match(img_src1, kp1, des1, img_src2, kp2, des2, embed=1, pt_flag=pt_flag,MIN_MATCH_COUNT=3)
	cv2.imshow('reu',reu_img)
	cv2.imwrite('reu.tif',reu_img)
	'''
	最初cv2.imshow方法弹出的图像显示窗口无响应，查阅资料后增加以下办法，
	通过调用destroyWindows()函数来释放由OpenCV创建的所有窗口
	'''
	cv2.waitKey(1000)
	cv2.destroyAllWindows()
